In [ ]:
import sqlite3
import pandas as pd

# Step 1: Load CSV into a DataFrame
csv_file = "sqf-2023.csv"
df = pd.read_csv(csv_file)

# Step 2: Create an SQLite database in memory
conn = sqlite3.connect(":memory:")  # Use ":memory:" for a temporary in-memory database
df.to_sql("my_table", conn, index=False, if_exists="replace")  # Save DataFrame to SQLite table

# Step 3: Query the data using SQL
query = "SELECT * FROM my_table LIMIT 10;"  # Example query
result = pd.read_sql_query(query, conn)

# Step 4: Display the result
print(result)

# Close the connection when done
conn.close()


: 